# 03 — Golden Ticket Scoring | SweetReturns Golden City

Implements the **5-tier Golden Ticket system** + **Platinum detection**.

Each ticket is a binary signal based on quantitative thresholds derived from:
- Historical drawdowns
- Volume
- Volatility
- Skewness
- Relative performance

**Input:** `stock_features.parquet` (from notebook 02)

**Output:** `stock_tickets.parquet` with Golden Score (0–5) and Platinum flag

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_parquet("stock_features.parquet")
print(f"Loaded: {df.shape}, Tickers: {df['ticker'].nunique()}, Columns: {len(df.columns)}")

## Ticket I — Dip Ticket ("The Sour Candy Drop")

Qualifies when `drawdown_percentile > 0.80`.

This catches stocks that are experiencing a historically deep dip relative to their own history.

In [ ]:
df["ticket_1_dip"] = (df["drawdown_percentile"] > 0.80).astype(int)
print(f"Ticket I (Dip) - qualifying: {df['ticket_1_dip'].sum()} / {len(df)} ({df['ticket_1_dip'].mean():.1%})")
print(f"Unique tickers qualifying (ever): {df[df['ticket_1_dip']==1]['ticker'].nunique()}")

## Ticket II — Shock Ticket ("The Jawbreaker")

Qualifies when `drawdown_percentile > 0.85` AND `volume_percentile > 0.90` AND `vol_percentile > 0.85`.

This catches violent dips with extreme volume — true "jawbreaker" events.

In [ ]:
df["ticket_2_shock"] = (
    (df["drawdown_percentile"] > 0.85) &
    (df["volume_percentile"] > 0.90) &
    (df["vol_percentile"] > 0.85)
).astype(int)
print(f"Ticket II (Shock) - qualifying: {df['ticket_2_shock'].sum()} / {len(df)} ({df['ticket_2_shock'].mean():.1%})")

## Ticket III — Asymmetry Ticket ("The Fortune Cookie")

Qualifies when deep dip + positive forward skew + outsized upside potential vs limited downside.

In [ ]:
# Forward 60-day return
df["fwd_60d_return"] = df.groupby("ticker")["Close"].transform(
    lambda x: x.shift(-60) / x - 1
)

# Forward 60-day skew (of daily returns over next 60 days)
def compute_fwd_skew(group):
    ret = group["daily_return"].values
    fwd_skew = np.full(len(ret), np.nan)
    for i in range(len(ret) - 60):
        window = ret[i+1:i+61]
        valid = window[~np.isnan(window)]
        if len(valid) >= 20:
            fwd_skew[i] = stats.skew(valid)
    group["fwd_60d_skew"] = fwd_skew
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_fwd_skew)

# Forward return distribution percentiles
def compute_fwd_distribution(group):
    ret = group["daily_return"].values
    p5 = np.full(len(ret), np.nan)
    p25 = np.full(len(ret), np.nan)
    median = np.full(len(ret), np.nan)
    p75 = np.full(len(ret), np.nan)
    p95 = np.full(len(ret), np.nan)
    for i in range(len(ret) - 60):
        window = ret[i+1:i+61]
        valid = window[~np.isnan(window)]
        if len(valid) >= 20:
            p5[i] = np.percentile(valid, 5)
            p25[i] = np.percentile(valid, 25)
            median[i] = np.percentile(valid, 50)
            p75[i] = np.percentile(valid, 75)
            p95[i] = np.percentile(valid, 95)
    group["fwd_p5"] = p5
    group["fwd_p25"] = p25
    group["fwd_median"] = median
    group["fwd_p75"] = p75
    group["fwd_p95"] = p95
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_fwd_distribution)
print(f"Forward return features computed")
print(df[["fwd_60d_return", "fwd_60d_skew", "fwd_p5", "fwd_median", "fwd_p95"]].describe())

In [ ]:
# Median drawdown per ticker (for downside bound)
median_drawdown = df.groupby("ticker")["drawdown_pct"].transform("median").abs()

df["ticket_3_asymmetry"] = (
    (df["drawdown_percentile"] > 0.85) &
    (df["fwd_60d_skew"] > 0) &
    (df["fwd_p95"] > 2 * df["fwd_median"].abs()) &
    (df["fwd_p5"] > -median_drawdown)
).astype(int)
print(f"Ticket III (Asymmetry) - qualifying: {df['ticket_3_asymmetry'].sum()} / {len(df)} ({df['ticket_3_asymmetry'].mean():.1%})")

## Ticket IV — Relative Dislocation Ticket ("The Taffy Pull")

Qualifies when the stock has deeply underperformed SPY and shows mean-reverting behavior.

In [ ]:
# Relative return history percentile 15 threshold
rel_return_p15 = df.groupby("ticker")["relative_return_vs_spy"].transform(
    lambda x: x.quantile(0.15)
)

# Mean reversion score: negative autocorrelation suggests mean-reverting
df["ticket_4_dislocation"] = (
    (df["drawdown_percentile"] > 0.80) &
    (df["relative_return_vs_spy"] < rel_return_p15) &
    (df["autocorrelation_lag1"] < -0.1)
).astype(int)
print(f"Ticket IV (Dislocation) - qualifying: {df['ticket_4_dislocation'].sum()} / {len(df)} ({df['ticket_4_dislocation'].mean():.1%})")

## Ticket V — Structural Convexity Ticket ("The Golden Gummy Bear")

The rarest ticket — requires extreme drawdown + volume + underperformance + positive skew + favorable volatility regime.

In [ ]:
# Favorable volatility regime: VIX proxy - use realized vol of market
favorable_vol = df["regime_label"].isin(["bull_quiet", "bear_quiet"])

df["ticket_5_convexity"] = (
    (df["drawdown_percentile"] > 0.90) &
    (df["volume_percentile"] > 0.90) &
    (df["relative_return_vs_spy"] < rel_return_p15) &
    (df["fwd_60d_skew"] > 0.5) &
    favorable_vol
).astype(int)
print(f"Ticket V (Convexity) - qualifying: {df['ticket_5_convexity'].sum()} / {len(df)} ({df['ticket_5_convexity'].mean():.1%})")

In [ ]:
# Golden Score: sum of 5 tickets (0-5)
df["golden_score"] = (
    df["ticket_1_dip"] + df["ticket_2_shock"] + df["ticket_3_asymmetry"] +
    df["ticket_4_dislocation"] + df["ticket_5_convexity"]
)

# Rarity percentile: cross-sectional rank on each date
df["rarity_percentile"] = df.groupby("Date")["golden_score"].rank(pct=True)

# Platinum: golden_score >= 4 AND rarity_percentile > 0.98 AND extreme conditions
df["is_platinum"] = (
    (df["golden_score"] >= 4) &
    (df["rarity_percentile"] > 0.98) &
    (df["fwd_60d_skew"] > 1.0) &
    (df["relative_return_vs_spy"] < df.groupby("ticker")["relative_return_vs_spy"].transform(lambda x: x.quantile(0.05))) &
    df["regime_label"].isin(["bull_quiet", "bear_quiet"])
).astype(bool)

print(f"\n=== Golden Score Distribution ===")
print(df["golden_score"].value_counts().sort_index())
print(f"\nPlatinum stores: {df['is_platinum'].sum()} events across {df[df['is_platinum']]['ticker'].nunique()} tickers")

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle("Golden Ticket Distribution", fontsize=14, color="#FFD700")
plt.style.use("dark_background")

# Score distribution
score_counts = df.groupby("Date")["golden_score"].value_counts().unstack(fill_value=0).mean()
colors = ["#F5F5DC", "#7FFF00", "#00BFFF", "#FFD700", "#FF69B4", "#FF4500"]
axes[0].bar(score_counts.index, score_counts.values, color=colors[:len(score_counts)])
axes[0].set_xlabel("Golden Score")
axes[0].set_ylabel("Avg stocks per day")
axes[0].set_title("Score Distribution", color="#FFD700")

# Tickets over time
monthly = df.set_index("Date").resample("ME")[["ticket_1_dip", "ticket_2_shock", "ticket_3_asymmetry", "ticket_4_dislocation", "ticket_5_convexity"]].sum()
monthly.plot(ax=axes[1], linewidth=1.5)
axes[1].set_title("Ticket Activity Over Time", color="#FFD700")
axes[1].legend(fontsize=7)

# Platinum events
plat_monthly = df[df["is_platinum"]].set_index("Date").resample("ME").size()
axes[2].bar(plat_monthly.index, plat_monthly.values, color="#DAA520", width=25)
axes[2].set_title("Platinum Events", color="#FFD700")

plt.tight_layout()
plt.savefig("golden_tickets.png", dpi=150, facecolor="#1a1a2e", bbox_inches="tight")
plt.show()

In [ ]:
df.to_parquet("stock_tickets.parquet", index=False)
print(f"Saved stock_tickets.parquet: {df.shape}")
print(f"Ready for 04_forward_returns.ipynb!")

## Summary

| Ticket | Name | Key Condition | Rarity |
|--------|------|---------------|--------|
| I | Dip (Sour Candy Drop) | `drawdown_percentile > 0.80` | ~20% |
| II | Shock (Jawbreaker) | Drawdown + Volume + Volatility | ~2-5% |
| III | Asymmetry (Fortune Cookie) | Dip + Positive Skew + Upside | ~1-3% |
| IV | Dislocation (Taffy Pull) | Underperformance + Mean Reversion | ~2-5% |
| V | Convexity (Golden Gummy Bear) | Extreme Multi-Factor | <1% |

**Golden Score** = sum of 5 binary tickets (0–5)

**Platinum** = Golden Score >= 4 + top 2% rarity + extreme skew + deep underperformance + quiet regime

**Next:** `04_forward_returns.ipynb` — analyze realized forward returns conditioned on ticket signals